In [ ]:
import os, cv2, tensorflow as tf, numpy as np
from matplotlib import pyplot as plt
import imghdr

In [ ]:
data_dir = 'data'
image_exts = ['jpeg','jpg','png','bmp']

In [ ]:
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir,image_class)):
        image_path = os.path.join(data_dir,image_class,image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                os.remove(image_path)
                
        except Exception as e:
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)


In [ ]:
data = tf.keras.utils.image_dataset_from_directory(data_dir)

In [ ]:
data = data.map(lambda x,y: (x/255,y))

In [ ]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2) + 1
test_size = int(len(data)*.1) + 1

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [ ]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(tf.keras.layers.MaxPooling2D())

model.add(tf.keras.layers.Conv2D(32, (3,3), 1, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())

model.add(tf.keras.layers.Conv2D(16, (3,3), 1, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256,activation='relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [ ]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(),metrics=['accuracy'])

In [ ]:
hist = model.fit(train,epochs=20,validation_data=val)

In [ ]:
model.save('sfw_nswf_identifier.h5')

In [ ]:
model = tf.keras.models.load_model('sfw_nswf_identifier.h5')

In [ ]:
img1test = cv2.imread('ZZZZZZZZZ _123883326_852a3a31-69d7-4849-81c7-8087bf630251 (20).jpg')
resize = tf.image.resize(img1test, (256,256))
yhat = model.predict(np.expand_dims(resize/255,0))
yhat